In [3]:
%pip install stable-baselines3[extra]

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
INFO: pip is looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.5 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.5 MB 885.6 kB/s eta 0:00:04
   --------- ------------------------------ 0.8/3.5 MB 1.0 MB/s eta 0:00:03
   --

  You can safely remove it manually.
  You can safely remove it manually.


In [1]:
from pyboy import PyBoy
pyboy = PyBoy('ROMs/Pokemon_Blue.gb', window='SDL2')
pyboy.set_emulation_speed(2)
for step in range(1000):
    pyboy.tick()
    pass
pyboy.button('START')
for step in range(1000):
    pyboy.tick()
    pyboy.button('a')
    pass
pyboy.stop()


In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from pyboy import PyBoy

class PyBoyPokemonEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"]}

    def __init__(self, rom_path, render_mode="rgb_array"):
        super().__init__()
        self.p_p_action = None
        self.p_action = None
        self.pyboy = PyBoy(
            rom_path,
            window="null" if render_mode != "human" else "SDL2",
            debug=False
        )
        with open("roms/state_file.state", "rb") as f:
            self.pyboy.load_state(f)
        self.pyboy.set_emulation_speed(0)
        # Action space: Up, Down, Left, Right, A, B, Start, Select
        self.buttons = [
            "up", "down", "left", "right",
            "a", "b", "start", "select"
        ]
        self.action_space = spaces.Discrete(len(self.buttons))

        # Observation: raw Game Boy screen (160×144 RGB
        frame = self.pyboy.screen.ndarray 
        H, W, C = 144, 160, 3  # after dropping alpha, C=3
        self.observation_space = spaces.Box(low=0, high=255, shape=(H, W, C), dtype=np.uint8)
        self.render_mode = render_mode

    def step(self, action):
        # Press selected button for a few frames
        screen = self.pyboy.game_area()

        button = self.buttons[action]
        self.pyboy.button_press(button)
        self.pyboy.tick(4)
        self.pyboy.button_release(button)
        self.pyboy.tick(16)
        
        # TODO: Define a meaningful reward function
        # if some change on the screen:
        
        try:
            test = np.abs(np.sum(screen) - np.sum(self.pyboy.game_area())) > 1000
        except OverflowError:
            test = True

        if test:
            reward = 1.0  # reward for screen change
        elif action == self.p_p_action and action == self.p_action:
                reward = -1.0  # small reward for repeating the same action
        else:
            reward = -0.1  # default reward

        terminated = False
        truncated = False
        self.p_p_action = self.p_action
        self.p_action = action
        return self._get_frame(), reward, terminated, truncated, {}
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # Reset emulator
        obs = self._get_frame()
        return obs, {}

    def render(self):
        if self.render_mode == "human":
            pass  # PyBoy manages its own SDL window

    def _get_frame(self):
        frame = self.pyboy.screen.ndarray[..., :3]  # keep only RGB
        return frame.astype(np.uint8)



In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecTransposeImage

env = PyBoyPokemonEnv("roms/Pokemon_Blue.gb", render_mode="human")
# vec_env = DummyVecEnv([lambda: env])
def make_env():
    return PyBoyPokemonEnv("roms/Pokemon_Blue.gb", render_mode="rgb_array")
env = DummyVecEnv([make_env for _ in range(4)])
# env = VecTransposeImage(env)

model = PPO(
    "CnnPolicy",
    env,
    verbose=1,
    tensorboard_log="./ppo_pokemon/"
)

model.learn(total_timesteps=10000)
model.save("ppo_pokemon_blue")
pyboy.stop()


Using cuda device
Wrapping the env in a VecTransposeImage.
Logging to ./ppo_pokemon/PPO_13


C:\Users\demje\AppData\Local\Temp\ipykernel_10428\1458923528.py:47: RuntimeWarning: overflow encountered in scalar subtract
  if np.abs(np.sum(screen) - np.sum(self.pyboy.game_area())) > 1000:


-----------------------------
| time/              |      |
|    fps             | 264  |
|    iterations      | 1    |
|    time_elapsed    | 30   |
|    total_timesteps | 8192 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 205         |
|    iterations           | 2           |
|    time_elapsed         | 79          |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.017075565 |
|    clip_fraction        | 0.271       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.06       |
|    explained_variance   | -0.0527     |
|    learning_rate        | 0.0003      |
|    loss                 | 1.79        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0282     |
|    value_loss           | 3.44        |
-----------------------------------------


In [4]:
# Run a single environment for visualization
from pyboy import PyBoy
from stable_baselines3 import PPO

model = PPO.load("./ppo_pokemon_blue.zip") 
visual_env = PyBoyPokemonEnv("roms/Pokemon_Blue.gb", render_mode="human")
obs, info = visual_env.reset()
print(f"Starting....")
for _ in range(10000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = visual_env.step(action)
    visual_env.render()  # SDL2 window updates
    if terminated or truncated:
        obs, info = visual_env.reset()

visual_env.pyboy.stop()


Starting....
